# Exploring Hacker News Posts

In this project, we'll compare two different types of posts from [Hacker News](https://news.ycombinator.com/), a popular site where technology related stories (or 'posts') are voted and commented upon. The two types of posts we'll explore begin with either `Ask HN` or `Show HN`.

Users submit `Ask HN` posts to ask the Hacker News community a specific question, such as "What is the best online course you've ever taken?" Likewise, users submit `Show HN` posts to show the Hacker News community a project, product, or just generally something interesting.

We'll specifically compare these two types of posts to determine the following:

- Do `Ask HN` or `Show HN` receive more comments on average?
- Do posts created at a certain time receive more comments on average?

It should be noted that the data set we're working with was reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions.

## Introduction

First, we'll read in the data and remove the headers.

In [1]:
from csv import reader

opened_file = open('./hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

## Removing Headers from a List of Lists

In [2]:
# Remove the headers
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


We can see above that the data set contains the title of the posts, the number of comments for each post, and the date the post was created. Let's start by exploring the number of comments for each type of post.

## Extracting `Ask HN` and `Show HN` Posts

First, we'll identify posts that begin with either `Ask HN` or `Show HN` and separate the data for those two types of posts into different lists. Separating the data makes it easier to analyze in the following steps.

In [3]:
# Identify posts that begin with either `Ask HN` or `Show HN` and separate the data into different lists
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
	title = row[1].lower()
	if title.startswith('ask'):
		ask_posts.append(row)
	elif title.startswith('show'):
		show_posts.append(row)
	else:
		other_posts.append(row)

print('ASK posts found:', len(ask_posts))
print('SHOW posts found:', len(show_posts))
print('Other posts found:', len(other_posts))

ASK posts found: 1756
SHOW posts found: 1164
Other posts found: 17180


## Calculating the Average Number of Comments for `Ask HN` and `Show HN` Posts

Now that we separated ask posts and show posts into different lists, we'll calculate the average number of comments each type of post receives.

In [4]:
# Calculate the average number of comments `Ask HN` posts receive
total_ask_comments = 0
for row in ask_posts:
	comments_number = int(row[4])
	total_ask_comments += comments_number
avg_ask_comments = total_ask_comments / len(ask_posts)
print('Average ASK comments:', avg_ask_comments)

Average ASK comments: 14.203302961275627


In [5]:
# Calculate the average number of comments `Show HN` posts receive
total_show_comments = 0
for row in show_posts:
	comments_number = int(row[4])
	total_show_comments += comments_number
avg_show_comments = total_show_comments / len(show_posts)
print('Average SHOW comments:', avg_show_comments)

Average SHOW comments: 10.323024054982818


On average, `Ask HN` posts in our sample receive approximately 14 comments, whereas `Show HN` posts receive approximately 10. Since `Ask HN` posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

## Finding the Amount of Ask Posts and Comments by Hour Created

Next, we'll determine if we can maximize the amount of comments an `Ask HN` post receives by creating it at a certain time. First, we'll find the amount of `Ask HN` posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments `Ask HN` posts created at each hour of the day receive.

In [6]:
# Calculate the amount of 'Ask HN' posts created during each hour of day and the number of comments received
import datetime as dt

result_list = []

for row in ask_posts:
	created_at = row[6]
	comments_number = int(row[4])
	result_list.append([created_at, comments_number])

posts_by_hour = {}
comments_by_hour = {}
date_format = '%m/%d/%Y %H:%M'

for row in result_list:
	date = row[0]
	comments_number = row[1]
	date_dt = dt.datetime.strptime(date, date_format)
	time = date_dt.strftime('%H')

	if time not in posts_by_hour:
		posts_by_hour[time] = 1
		comments_by_hour[time] = comments_number
	else:
		posts_by_hour[time] += 1
		comments_by_hour[time] += comments_number

print('Posts by hour:', posts_by_hour)
print('Comments by hour:', comments_by_hour)

Posts by hour: {'09': 45, '13': 86, '10': 60, '14': 110, '16': 109, '23': 70, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 55, '05': 46, '19': 112, '01': 60, '22': 71, '08': 48, '04': 48, '00': 55, '06': 44, '07': 34, '11': 58}
Comments by hour: {'09': 251, '13': 1254, '10': 794, '14': 1420, '16': 1949, '23': 713, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 459, '05': 464, '19': 1295, '01': 683, '22': 479, '08': 492, '04': 339, '00': 447, '06': 397, '07': 267, '11': 641}


## Calculating the Average Number of Comments for `Ask HN` Posts by Hour

In [7]:
# Calculate the average amount of comments `Ask HN` posts created at each hour of the day receive
avg_comments_by_hour = []

for each_hour in comments_by_hour:
	avg_comments_by_hour.append([each_hour, comments_by_hour[each_hour] / posts_by_hour[each_hour]])

avg_comments_by_hour

[['09', 5.5777777777777775],
 ['13', 14.581395348837209],
 ['10', 13.233333333333333],
 ['14', 12.909090909090908],
 ['16', 17.880733944954127],
 ['23', 10.185714285714285],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 8.345454545454546],
 ['05', 10.08695652173913],
 ['19', 11.5625],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.0625],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

## Sorting and Printing Values from a List of Lists

In [8]:
swap_avg_comments_by_hour = []
for row in avg_comments_by_hour:
	swap_avg_comments_by_hour.append([row[1], row[0]])

print(swap_avg_comments_by_hour)

sorted_swap = sorted(swap_avg_comments_by_hour, reverse=True)
sorted_swap

[[5.5777777777777775, '09'], [14.581395348837209, '13'], [13.233333333333333, '10'], [12.909090909090908, '14'], [17.880733944954127, '16'], [10.185714285714285, '23'], [9.41095890410959, '12'], [11.46, '17'], [38.5948275862069, '15'], [16.009174311926607, '21'], [21.525, '20'], [23.810344827586206, '02'], [13.20183486238532, '18'], [8.345454545454546, '03'], [10.08695652173913, '05'], [11.5625, '19'], [11.383333333333333, '01'], [6.746478873239437, '22'], [10.25, '08'], [7.0625, '04'], [8.127272727272727, '00'], [9.022727272727273, '06'], [7.852941176470588, '07'], [11.051724137931034, '11']]


[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [17.880733944954127, '16'],
 [16.009174311926607, '21'],
 [14.581395348837209, '13'],
 [13.233333333333333, '10'],
 [13.20183486238532, '18'],
 [12.909090909090908, '14'],
 [11.5625, '19'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.25, '08'],
 [10.185714285714285, '23'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.345454545454546, '03'],
 [8.127272727272727, '00'],
 [7.852941176470588, '07'],
 [7.0625, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [10]:
# Sort the values and print the the 5 hours with the highest average comments
print('Top 5 Hours for ASK Posts Comments:')

for row in sorted_swap[:5]:
	time = dt.datetime.strptime(row[1], '%H').strftime('%H:%M')
	comments_number = row[0]
	print(f'{time}: {comments_number:.2f} average comments per post')

Top 5 Hours for ASK Posts Comments:
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 17.88 average comments per post
21:00: 16.01 average comments per post


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

## Conclusion

In this project, we analyzed `Ask HN` posts and `Show HN` posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as `Ask HN` post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that _of the posts that received comments_, ask posts received more comments on average and `Ask HN` posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.